In [11]:
from abc import ABC, abstractmethod
import networkx as nx
import numpy as np

class Plannifier(ABC):

    def __init__(self, domain):
        self.domain = domain
        self.actions = domain.action_dic
    
    @abstractmethod
    def solve(self):
        pass
    
    
class AlphaStarPlannifier(Plannifier):
    # TODO : we could handle the ancestors through an attribute of the class ancestors = Dict(State,List(State))
    
    def __init__(self, domain, dist_fn, heuristic_fn):
        Plannifier().__init__(self, domain)
        self.heuristic_fn = heuristic_fn # S -> R : heuristic distance to end
        self.dist_fn = dist_fn # S x S -> R : distance between the states
        # list of (node, distance_to_start, value, ancestors) ordered by ascending value
        # (value = distance_to_start + heuristic_to_end)
        self.nodes = []
        
    def solve(self):
        # TODO : extract iniState from the domain / we coudl also just pass a iniState param in 
        # __init__ since self.domain is never used
        init = self.domain.init_state 
        # initialize tree search with init state, distance to start 0, heuristic to end, 
        # empty list of moves to get there
        self.nodes = [init, 0, self.heuristic_fn(init), []] 
        solved = init.isFinal()
        while not solved:
            new_nodes, ancestors = self.expand()
            for node, ancestors in zip(new_nodes, ancestors):
                if node.isFinal():
                    return ancestors
        print('No plan found !')
        return
        
    def expand(self):
        """finds the best node and adds its children to the stack of considered states
        returns: the list of added children"""
        best_node, dist, value, ancestors = self.nodes.pop() # get our best current node
        children = best_node.get_children(self.actions)
        children_ancestors = []
        for move_type, move_args, child in children:
            move = (move_type, move_args)
            child_ancestors = ancestors + [move]
            # g(n) : distance to start = distance of parent to start + distance of child to parent
            child_dist += dist + self.dist_fn(best_node, child)
            # h(n) : (estimated) distance to end
            child_heur = self.heuristic_fn(child)
            child_value = child_dist + child_heur
            self.insert_node(child, child_dist, child_value, ancestors) 
            children_ancestors.append(child_ancestors)
        return children, children_ancestors
        
    def insert_node(self, node, dist, value, ancestors):
        """insert (node,value,ancestors) in the list of considered states and returns the index where it was inserted"""
        idx = 0
         # handle extreme case that will cause problems with the while
        if value >= self.nodes[-1][2]:
            self.nodes.append((node,dist, value, ancestors))
            return len(self.nodes)-1
        # find index where to insert the node
        while self.nodes[idx][1] < value:
            idx += 1
        self.nodes.insert(idx,(node,dist, value, ancestors))
        return idx

In [12]:
def distance_one(from_, to_):
    return 1